In [2]:
cd("/home/rafael/UNAM/Sanders/Mancuerna/Dumbbell/Dumbbell-Julia/Codigo-julia")

include("dumbbell_type.jl")
include("Gram_Schmidt.jl")
include("dumbbell_lyapunov_functions.jl")


evol_δΓ (generic function with 3 methods)

In [3]:
walls=reshape([[-3, 3], [-2, 2]], 2, 2)

2x2 Array{Int32,2}:
 -3  -2
  3   2

In [4]:
hw = walls[1:2]
vw = walls[3:4]

hw, vw

([-3,3],[-2,2])

disp_vecs_transformation (generic function with 1 method)

In [30]:
function disp_vecs_transformation(db::dumbbell, walls, norm_disp_vecs)
    
    ### Given a state of the dumbbell, 'db', inside thw billiard with frontiers, 'walls,
    ### And a set of normalized displacement vectors, 'norm_disp_vecs',
    ### This function will calculate the evolution of the displacement vectors (DV); their transformation
    ### during the collision, and a new set of orthogonal and orthonormal vectors: 'ortho_disp_vecs'; 
    ### norm_disp_vecs2.
    ### The output is a tuple of the collision time ('tc') it took the dumbbell to reach for the next wall,
    ### ortho_disp_vecs, and norm_disp_vecs2
    
    vert_walls = walls[1:2]; horiz_walls = walls[3:4] # coordinates of the horizontal and vertical walls, respectively 

    Γ0,  Γf, wall, part, ct = collision(db, vert_walls, horiz_walls)
    
    disp_vecs0 = Array{Float64}[]
    for v in norm_disp_vecs
        vf = evol_δΓ(v, ct, l0, m0) # evolution of each of the initial normalized DV
        push!(disp_vecs0, [vf])
    end

    disp_vecs_F = Array{Float64}[]
    for dΓ in disp_vecs0
        dΓp = displacement_vector_collision_map(Γ0, dΓ, part, wall, l0, m0)
        push!(disp_vecs_F, [dΓp])
    end
    
    ortho_disp_vecs, norm_disp_vecs2 = Gram_Schmidt(disp_vecs_F)
    
    ct, ortho_disp_vecs, norm_disp_vecs2
end

function lyapunov_spectrum(db::dumbbell, walls, N)
    # N=number of collisions
    srand(5) #seed for random number generator  
    
    rand_vecs = ([rand(6) for i in 1:6]) # initial random displacement vectors
    ortho_disp_vecs, norm_disp_vecs = Gram_Schmidt(rand_vecs)  # initial DV, orthogonalized and orthonormalized
    
    col_count = 0 # collision counter

    λ_col = zeros(Float64,N,6); λ_time = zeros(Float64,N,6) #value of the LEs
    χs = zeros(Float64,N,6) # log of the norm of (the sum of) the DV
    
    # arrays of the collision time and the total elapsed time (until each collision)
    col_times = Float64[]; elapsed_time = Float64[]; 
    t_T= 0. # initial time
    
    # main loop for calculating the LEs for the N collisions
    for k in 1:N
        col_count += 1
        
        #collision time, orthogonal vectors, orthonormal vectors after the k-th collision
        ct, O_vecs, ON_vecs = disp_vecs_transformation(db, walls, norm_disp_vecs)
        
        t_T += ct # add the current collision time to the elapsed time
        push!(elapsed_time, t_T);  push!(col_times, ct) # register the times
        
        for (j,ov) in enumerate(O_vecs)
            if k>1
                χs[k, j] = χs[k-1, j] + log(norm(ov))
            else
                χs[1,j] = log(norm(ov))
            end
            
            # LEs obtained with the number of collisions and the elapsed time 
            λ_col[k,j] = χs[k,j]/(k); λ_time[k,j] = χs[k,j]/t_T 
        end
        
        norm_disp_vecs = ON_vecs # set the new DV as the normalized DV after the collision
        
    end
    
    λ_col, λ_time, elapsed_time, col_times
end

lyapunov_spectrum (generic function with 1 method)

In [31]:
rcm = [1., 1.]; l0=0.2; m0=1.; vcm = [2.1, 0.6]; θ=π/6; ω=2.

db = dumbbell(rcm, θ, vcm, ω, m0, l0, 0)

walls = [[-3,3], [-2, 2]]
nc=10

lyapunov_spectrum(db, walls, nc)

(
10x6 Array{Float64,2}:
 2.25869  1.85146   0.389209   -0.778905   -0.249198  -3.47125
 1.62169  1.159     0.554522   -0.38981    -0.913962  -2.03144
 2.22333  1.57687   0.566597   -0.310255   -1.84241   -2.21413
 1.52041  0.783152  0.425829   -0.172616   -1.1008    -1.45598
 1.96325  0.977456  0.421425   -0.257026   -1.22746   -1.87764
 1.75712  1.0264    0.6871     -0.237708   -1.04723   -2.18568
 1.92084  1.16304   0.264365   -0.166461   -1.07624   -2.10554
 1.91366  1.04765   0.0873642  -0.0589306  -0.779537  -2.21021
 1.54265  0.959909  0.120417   -0.0491905  -0.601016  -1.97277
 1.48107  0.835131  0.118432    0.098121   -0.682031  -1.85072,

10x6 Array{Float64,2}:
 2.54118  2.08301   0.437886   -0.87632    -0.280364  -3.90539
 2.14366  1.53205   0.733006   -0.515279   -1.20814   -2.6853 
 1.90313  1.34977   0.484997   -0.265573   -1.57707   -1.89526
 1.72706  0.889599  0.483708   -0.196078   -1.25042   -1.65387
 1.78762  0.890013  0.383724   -0.234033   -1.11765   -1.70967
 1.58

In [11]:
srand(5)